# Part I. ETL Pipeline for Pre-Processing the Files

### PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# delete .ipynb_checkpoints file in event_data folder in terminal using below commands

# ls -al /home/workspace/event_data
# rm -r /home/workspace/event_data/.ipynb_checkpoints
# ls -al /home/workspace/event_data

In [2]:
# check current working directory
print(os.getcwd())

# get current folder and subfolder event data
filepath = os.getcwd() + '/event_data/'

# create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*.csv'))
    
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiate an empty list of rows that will be generated from each file
full_data_rows_list = [] 

# for every filepath in the file path list 
for f in file_path_list:

# read csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extract each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
print(len(full_data_rows_list))

# show list of event data rows
# print(full_data_rows_list)

# create a smaller event data csv file called event_datafile_full csv that will be used \
# to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    # add column titles to be added in the new csv file
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                        'level','location','sessionId','song','userId'])
    # assign column elements in original full_data_rows_list for columns in the new csv accordingly
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], \
                         row[7], row[8], row[12], row[13], row[16]))

8056


In [4]:
# check number of rows in the new csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Data Modeling with Apache Cassandra

#### The newly created CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory, contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Create Cluster

In [5]:
# connect to locally installed Cassandra instance (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# establish connection and begin executing queries with session
session = cluster.connect()

#### Create Keyspace

In [6]:
# create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# set KEYSPACE to the created keyspace
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Create tables and queries to anwser the following three questions: 

##### 1. Give the artist, song title and song's length in the music app history that was heard during given sessionId and with certain itemInSession


##### 2. Give the name of artist, song (sorted by itemInSession) and user (first and last name) for given userid and sessionid
    

##### 3. Give every user name (first and last) in my music app history who listened to a given song

#### Question 1

##### Create table song to provide song info including artist name, song name, and song length according to user listening sessionId and itemInSession. This table has sessionId as the partition key and a clustering column itemInSession

In [8]:
# create table for question 1
query = "create table if not exists song"
query = query + "(sessionId int, itemInSession int, artist text, song text, length decimal, \
                    primary key (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# use the new csv file to load data into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # assign the INSERT statements into the `query` variable
        query = "INSERT INTO song (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # assign column elements for each column in the INSERT statement
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
# execute SELECT statement to verify the data was entered into the table
query = "select artist, song, length from song where sessionId = 338 and itemInSession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, ', ', row.song, ', ', row.length)

Faithless ,  Music Matters (Mark Knight Dub) ,  495.3073


#### Question 2

##### Create table user_song to provide artist name and song name users listened to with user names according to userId, sessionId, and itemInSession. This table has both userId and sessionId as the partition keys and a clustering column itemInSession which sorts data descendingly

In [11]:
# create table for question 2
query = "create table if not exists user_song"
query = query + "(userId int, sessionId int, itemInSession int, \
                    artist text, song text, firstName text, lastName text, \
                    primary key ((userId, sessionId), itemInSession)) \
                with clustering order by (itemInSession DESC)"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
# use the new csv file to load data into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # add the INSERT statements into the `query` variable
        query = "INSERT INTO user_song (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # assign column elements for each column in the INSERT statement
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
# execute SELECT statement to verify the data was entered into the table
query = "select artist, song, firstName, lastName from user_song where userid = 10 and sessionid = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, ', ', row.song, ', ', row.firstname, row.lastname)

Lonnie Gordon ,  Catch You Baby (Steve Pitron & Max Sanna Radio Edit) ,  Sylvie Cruz
Sebastien Tellier ,  Kilometer ,  Sylvie Cruz
Three Drives ,  Greece 2000 ,  Sylvie Cruz
Down To The Bone ,  Keep On Keepin' On ,  Sylvie Cruz


#### Question 3

##### Create table user to provide user names according to the song users listened to during which sessionId. This table has song name as the partition key and a clustering column sessionId

In [14]:
# create table for question 3
query = "create table if not exists user"
query = query + "(song text, sessionId int, firstName text, lastName text, \
                    primary key (song, sessionId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [16]:
# use the new csv file to load data into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # add the INSERT statements into the `query` variable
        query = "INSERT INTO user (song, sessionId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        # assign column elements for each column in the INSERT statement
        session.execute(query, (line[9], int(line[8]), line[1], line[4]))

In [17]:
# execute SELECT statement to verify the data was entered into the table
query = "select firstName, lastName from user where song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Sara Johnson
Jacqueline Lynch
Tegan Levine


### Drop all the created tables before closing out the sessions

In [18]:
query = "drop table if exists song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists user_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()